## FIN 5350 - Homework 2

In [92]:
import numpy as np
from scipy.stats import binom

def callPayoff(spot, strike):
    return np.maximum(spot - strike, 0.0)

def putPayoff(spot, strike):
    return np.maximum(strike - spot, 0.0)

def euroBinomPricerRecursiveMatrix(S, K, r, v, q, T, n, payoff, verbose = True):
    nodes = n  + 1
    h = T / n
    u = 1.3
    d = 0.8
    pu = (np.exp((r - q) * h) - d) / (u - d)
    pd = 1.0 - pu
    disc = np.exp(-r * h)
    
    Ct = np.zeros((nodes, n+1))
    St = np.zeros((nodes, n+1))
    Dt = np.zeros((nodes, n+1))
    Bt = np.zeros((nodes, n+1))
    
    for i in range(nodes):
        St[i, n] = S * (u**(n-i)) * (d**i)
        Ct[i, n] = payoff(St[i, n], K)
    

    for t in range((n-1), -1, -1):
        for j in range(t+1):
            St[j, t] = St[j, t+1] / u
            Ct[j, t] = disc * ((pu * Ct[j, t+1]) + (pd * Ct[j+1, t+1]))
            D = Dt[j, t] = np.exp(-q * h) * (Ct[j, t+1] - Ct[j+1, t+1]) / (St[j, t] * (u - d))
            B = Bt[j, t] = np.exp(-r * h) * (((u * Ct[j + 1, t+1]) - (d * Ct[j, t+ 1]))/(u - d))
            
        
    if verbose:
        print(np.around(St, 3))
        print("\n")
        print(np.around(Ct, 3))
        print("\n")
        print(np.around(Dt, 3))
        print("\n")
        print(np.around(Bt, 3))
                    
            
    return Ct[0,0]

##### __Problem 1__
 
##### Let $S = \$100$, $K = \$105$, $r = 8\%$, $T = 0.5$, and $\delta = 0.0$. Let $u = 1.3$, $d = 0.8$, and $n = 1$.

##### a. What are the premium, $\Delta$, and $B$ for a European call?

In [81]:
S = 100.0
K = 105.0
r = 0.08
q = 0.0
T = 0.5
n = 1.0
h = T / n
u = 1.3
d = 0.8

fu = callPayoff(u * S, K)
fd = callPayoff(d * S, K)
D = (fu - fd)/(S * (u - d))
B = np.exp(-r * h) * ((u * fd - d * fu)/ (u - d))
f = S * D + B

print(f"The Call Premium is: {f:0.4f}")
print(f"Delta is: {D}")
print(f"B is: {B:0.4f}")

The Call Premium is: 11.5684
Delta is: 0.5
B is: -38.4316


##### b. What are the premium, $\Delta$, and $B$ for a European put?

In [82]:
fu = putPayoff(u * S, K)
fd = putPayoff(d * S, K)
D = (fu - fd)/(S * (u - d))
B = np.exp(-r * h) * ((u * fd - d * fu)/ (u - d))
f = S * D + B
print(f"The Put Premium is: {f:0.4f}")
print(f"Delta is: {D}")
print(f"B is: {B:0.4f}")

The Put Premium is: 12.4513
Delta is: -0.5
B is: 62.4513


##### __Problem 2__

##### Let $S = \$100$, $K = \$95$, $r = 8\%$, $T = 0.5$, and $\delta = 0.0$. Let $u = 1.3$, $d = 0.8$, and $n = 1$.

##### a. Verify that the price of a European put is $\$7.471$.

In [83]:
S = 100.0
K = 95.0
r = 0.08
q = 0.0
T = 0.5
n = 1.0
h = T / n
u = 1.3
d = 0.8

fu = putPayoff(u * S, K)
fd = putPayoff(d * S, K)
D = (fu - fd)/(S * (u - d))
B = np.exp(-r * h) * ((u * fd - d * fu)/ (u - d))
f = S * D + B
print(f"The Put Premium is: {f:0.4f}")

The Put Premium is: 7.4708


##### b. Suppose you observe a call price of $\$17$. What is the arbitrage?

In [84]:
fu = callPayoff(u * S, K)
fd = callPayoff(d * S, K)
D = (fu - fd)/(S * (u - d))
B = np.exp(-r * h) * ((u * fd - d * fu)/ (u - d))
f = S * D + B
print(f"The arbitrage is: {np.maximum(17-f, f-17):0.4f}")

The arbitrage is: 0.8042


##### c. Suppose you observe a call price of $\$15.50$. What is the arbitrage?

In [85]:
print(f"The arbitrage is: {np.maximum(15.50-f, f-15.50):0.4f}")

The arbitrage is: 0.6958


##### __Problem 3__

##### Let $S = \$100$, $K = \$95$, $\sigma = 30\%$, $r = 8\%$, $T = 1$, and $\delta = 0.0$. Let $u = 1.3$, $d = 0.8$, and $n = 2$. Construct the binomial tree for a call option. At each node provide the premium, $\Delta$, and $B$.

In [101]:
S = 100
K = 95
r = 0.08
v = 0.30
q = 0.0
T = 1
n = 2

print(f"1. Spot Price\n2. Call Price\n3. Delta\n4. B")
print("\n")
callPrc = euroBinomPricerRecursiveMatrix(S, K, r, v, q, T, n, callPayoff,  verbose = True)
print("\n")
print(f"The Call Premium is: {callPrc : 0.4f}")

1. Spot Price
2. Call Price
3. Delta
4. B


[[100. 130. 169.]
 [  0.  80. 104.]
 [  0.   0.  64.]]


[[19.994 38.725 74.   ]
 [ 0.     4.165  9.   ]
 [ 0.     0.     0.   ]]


[[0.691 1.    0.   ]
 [0.    0.225 0.   ]
 [0.    0.    0.   ]]


[[-49.127 -91.275   0.   ]
 [  0.    -13.835   0.   ]
 [  0.      0.      0.   ]]


The Call Premium is:  19.9937


##### __Problem 4__

Repeat the option price calculation in the previous question for stock prices of $\$80$, $\$90$, $\$110$,
$\$120$, and $\$130$, but now let $n = 3$. Keep everyting else fixed. What happens to the initial option
$\Delta$ as the stock price increases?

In [102]:
S = 80
K = 95
r = 0.08
v = 0.30
q = 0.0
T = 1
n = 3

print(f"1. Spot Price\n2. Call Price\n3. Delta\n4. B")
print("\n")
callPrc = euroBinomPricerRecursiveMatrix(S, K, r, v, q, T, n, callPayoff,  verbose = True)
print("\n")
print(f"The Call Premium is: {callPrc : 0.4f}")

1. Spot Price
2. Call Price
3. Delta
4. B


[[ 80.   104.   135.2  175.76]
 [  0.    64.    83.2  108.16]
 [  0.     0.    51.2   66.56]
 [  0.     0.     0.    40.96]]


[[11.081 21.971 42.7   80.76 ]
 [ 0.     2.572  5.818 13.16 ]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]]


[[0.485 0.709 1.    0.   ]
 [0.    0.182 0.316 0.   ]
 [0.    0.    0.    0.   ]
 [0.    0.    0.    0.   ]]


[[-27.716 -51.793 -92.5     0.   ]
 [  0.     -9.064 -20.502   0.   ]
 [  0.      0.      0.      0.   ]
 [  0.      0.      0.      0.   ]]


The Call Premium is:  11.0806


In [103]:
S = 90
K = 95
r = 0.08
v = 0.30
q = 0.0
T = 1
n = 2

print(f"1. Spot Price\n2. Call Price\n3. Delta\n4. B")
print("\n")
callPrc = euroBinomPricerRecursiveMatrix(S, K, r, v, q, T, n, callPayoff,  verbose = True)
print("\n")
print(f"The Call Premium is: {callPrc : 0.4f}")

1. Spot Price
2. Call Price
3. Delta
4. B


[[ 90.  117.  152.1]
 [  0.   72.   93.6]
 [  0.    0.   57.6]]


[[12.227 26.422 57.1  ]
 [ 0.     0.     0.   ]
 [ 0.     0.     0.   ]]


[[0.587 0.976 0.   ]
 [0.    0.    0.   ]
 [0.    0.    0.   ]]


[[-40.618 -87.778   0.   ]
 [  0.      0.      0.   ]
 [  0.      0.      0.   ]]


The Call Premium is:  12.2266


In [104]:
S = 110
K = 95
r = 0.08
v = 0.30
q = 0.0
T = 1
n = 2

print(f"1. Spot Price\n2. Call Price\n3. Delta\n4. B")
print("\n")
callPrc = euroBinomPricerRecursiveMatrix(S, K, r, v, q, T, n, callPayoff,  verbose = True)
print("\n")
print(f"The Call Premium is: {callPrc : 0.4f}")

1. Spot Price
2. Call Price
3. Delta
4. B


[[110.  143.  185.9]
 [  0.   88.  114.4]
 [  0.    0.   70.4]]


[[28.406 51.725 90.9  ]
 [ 0.     8.977 19.4  ]
 [ 0.     0.     0.   ]]


[[0.777 1.    0.   ]
 [0.    0.441 0.   ]
 [0.    0.    0.   ]]


[[-57.09  -91.275   0.   ]
 [  0.    -29.823   0.   ]
 [  0.      0.      0.   ]]


The Call Premium is:  28.4061


In [105]:
S = 120
K = 95
r = 0.08
v = 0.30
q = 0.0
T = 1
n = 2

print(f"1. Spot Price\n2. Call Price\n3. Delta\n4. B")
print("\n")
callPrc = euroBinomPricerRecursiveMatrix(S, K, r, v, q, T, n, callPayoff,  verbose = True)
print("\n")
print(f"The Call Premium is: {callPrc : 0.4f}")

1. Spot Price
2. Call Price
3. Delta
4. B


[[120.  156.  202.8]
 [  0.   96.  124.8]
 [  0.    0.   76.8]]


[[ 36.819  64.725 107.8  ]
 [  0.     13.79   29.8  ]
 [  0.      0.      0.   ]]


[[0.849 1.    0.   ]
 [0.    0.621 0.   ]
 [0.    0.    0.   ]]


[[-65.052 -91.275   0.   ]
 [  0.    -45.81    0.   ]
 [  0.      0.      0.   ]]


The Call Premium is:  36.8186


In [106]:
S = 130
K = 95
r = 0.08
v = 0.30
q = 0.0
T = 1
n = 2

print(f"1. Spot Price\n2. Call Price\n3. Delta\n4. B")
print("\n")
callPrc = euroBinomPricerRecursiveMatrix(S, K, r, v, q, T, n, callPayoff,  verbose = True)
print("\n")
print(f"The Call Premium is: {callPrc : 0.4f}")

1. Spot Price
2. Call Price
3. Delta
4. B


[[130.  169.  219.7]
 [  0.  104.  135.2]
 [  0.    0.   83.2]]


[[ 45.231  77.725 124.7  ]
 [  0.     18.602  40.2  ]
 [  0.      0.      0.   ]]


[[0.91  1.    0.   ]
 [0.    0.773 0.   ]
 [0.    0.    0.   ]]


[[-73.015 -91.275   0.   ]
 [  0.    -61.798   0.   ]
 [  0.      0.      0.   ]]


The Call Premium is:  45.2310


###### Delta increases at a decreasing rate as the spot price increases.

##### __Problem 5__

Let $S = \$100$, $K = \$95$, $r = 8\%$ (continuously compounded), $\sigma = 30\%$, $\delta = 0$, and $T = 1$ year and $n = 3$.

##### a. What is the premium for an American call option? Is there any early exercise?

In [121]:
def BinomPricerRecursiveMatrix(S, K, r, v, q, T, n, payoff, verbose = True):
    nodes = n  + 1
    h = T / n
    u = np.exp((r - q) * h + v * np.sqrt(h))
    d = np.exp((r - q) * h - v * np.sqrt(h))
    pu = (np.exp((r - q) * h) - d) / (u - d)
    pd = 1.0 - pu
    disc = np.exp(-r * h)
    
    Ct = np.zeros((nodes, n+1))
    St = np.zeros((nodes, n+1))
    
    for i in range(nodes):
        St[i, n] = S * (u**(n-i)) * (d**i)
        Ct[i, n] = payoff(St[i, n], K)
    

    for t in range((n-1), -1, -1):
        for j in range(t+1):
            St[j, t] = St[j, t+1] / u
            Ct[j, t] = disc * ((pu * Ct[j, t+1]) + (pd * Ct[j+1, t+1]))
        
    if verbose:
        print(np.around(St, 3))
        print("\n")
        print(np.around(Ct, 3))
        print("\n")
                    
            
    return Ct[0,0]

In [124]:
S = 100
K = 95
r = 0.08
v = 0.30
q = 0.0
T = 1
n = 3

callPrc = BinomPricerRecursiveMatrix(S, K, r, v, q, T, n, callPayoff, verbose = True)
print(f"The American Call Premium is: {callPrc : 0.4f}")

[[100.    122.125 149.144 182.142]
 [  0.     86.369 105.478 128.815]
 [  0.      0.     74.596  91.101]
 [  0.      0.      0.     64.428]]


[[18.283 33.149 56.644 87.142]
 [ 0.     6.69  15.04  33.815]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]]


The American Call Premium is:  18.2826


###### There is no early exercise because there is no dividend.

##### b. What is the premium for a European call option? Use the computational shortcut with the risk-neutral binomial pmf that I showed you in class. Compare the American and European premia.

In [125]:
def euroBinomPricer(S, K, r, v, q, T, n, payoff, verbose = True):
    nodes = n  + 1
    h = T / n
    u = np.exp((r - q) * h + v * np.sqrt(h))
    d = np.exp((r - q) * h - v * np.sqrt(h))
    pstar = (np.exp((r - q) * h) - d) / (u - d)
    
    price = 0.0
    
    for i in range(nodes):
        prob = binom.pmf(i, n, pstar)
        spotT = S * (u ** i) * (d ** (n - i))
        po = payoff(spotT, K) 
        price += po * prob
        if verbose:
            print(f"({spotT:0.4f}, {po:0.4f}, {prob:0.4f})")
        
    price *= np.exp(-r * T)
    
    return price

In [126]:
callPrc = euroBinomPricer(S, K, r, v, q, T, n, callPayoff, verbose = True)
print(f"The European Call Premium is: {callPrc : 0.4f}")

(64.4284, 0.0000, 0.1603)
(91.1007, 0.0000, 0.4044)
(128.8147, 33.8147, 0.3400)
(182.1418, 87.1418, 0.0953)
The European Call Premium is:  18.2826


###### The American and European call premia are equivalent.

##### c. What is the premium for a European put? Does put-call parity hold? (see McDonald Chapter 9). Also use the risk-neutral binomial pmf for this problem.

In [127]:
putPrc = euroBinomPricer(S, K, r, v, q, T, n, putPayoff, verbose = True)
print(f"The European Put Premium is: {putPrc : 0.4f}")

(64.4284, 30.5716, 0.1603)
(91.1007, 3.8993, 0.4044)
(128.8147, 0.0000, 0.3400)
(182.1418, 0.0000, 0.0953)
The European Put Premium is:  5.9786


In [131]:
parity = S - (K * np.exp(-r*T))
print(parity)
callPrc - putPrc

12.3039470932696


12.303947093269588

###### Put-call parity holds. 12.3039 = 12.3039

##### d. What is the premium of the American put? Compare with the European put. If they differ, explain why.

In [142]:
def americanBinomPricerRecursive(S, K, r, v, q, T, n, payoff, verbose = True):
    nodes = n  + 1
    h = T / n
    u = np.exp((r - q) * h + v * np.sqrt(h))
    d = np.exp((r - q) * h - v * np.sqrt(h))
    pu = (np.exp((r - q) * h) - d) / (u - d)
    pd = 1.0 - pu
    disc = np.exp(-r * h)
    
    Ct = np.empty(nodes)
    St = np.empty(nodes)
    
    for i in range(nodes):
        St[i] = S * (u ** (n - i)) * (d ** i)
        Ct[i] = payoff(St[i], K)
    
    if verbose:
        print(Ct)
        
    for t in range((n - 1), -1, -1):
        for j in range(t+1):
            Ct[j] = disc * (pu * Ct[j] + pd * Ct[j+1])
            St[j] = St[j] / u
            Ct[j] = np.maximum(Ct[j], K - St[j])
            print(Ct)
            
    return Ct[0]

In [143]:
putPrc = americanBinomPricerRecursive(S, K, r, v, q, T, n, putPayoff, verbose = True)
print(f"The American Put Premium is: {putPrc : 0.4f}")

[ 0.          0.          3.16660277 11.16886174]
[ 0.          0.          3.16660277 11.16886174]
[ 0.          1.65788424  3.16660277 11.16886174]
[ 0.          1.65788424  7.84406148 11.16886174]
[ 0.86799019  1.65788424  7.84406148 11.16886174]
[ 0.86799019  4.8747172   7.84406148 11.16886174]
[ 2.95422756  4.8747172   7.84406148 11.16886174]
The American Put Premium is:  2.9542


###### The American put is worth more because there is the option for early exercise.

##### __Problem 6__

Let $S = \$40$, $K = \$40$, $r = 8\%$ (continuously compounded), $\sigma = 30\%$, $\delta = 0.0$, $T = 0.5$ year,
and $n = 3$.

In [141]:
S = 40
K = 40
r = 0.08
v = 0.30
q = 0.0
T = 0.5
n = 3
h = T/n

u6 = np.exp((r - q) * h + v * np.sqrt(h))
d6 = np.exp((r - q) * h - v * np.sqrt(h))

##### a. Construct the binomial tree for the stock. What are $u$ and $d$?

In [148]:
callPrc = BinomPricerRecursiveMatrix(S, K, r, v, q, T, n, callPayoff, verbose = True)
putPrc = BinomPricerRecursiveMatrix(S, K, r, v, q, T, n, putPayoff, verbose = True)
print(f"u is {u6:0.4f} and d is {d6:0.4f}")

[[40.    45.818 52.483 60.118]
 [ 0.    35.864 41.081 47.057]
 [ 0.     0.    32.156 36.833]
 [ 0.     0.     0.    28.831]]


[[ 4.377  7.739 13.013 20.118]
 [ 0.     1.514  3.269  7.057]
 [ 0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.   ]]


[[40.    45.818 52.483 60.118]
 [ 0.    35.864 41.081 47.057]
 [ 0.     0.    32.156 36.833]
 [ 0.     0.     0.    28.831]]


[[ 2.809  0.868  0.     0.   ]
 [ 0.     4.597  1.658  0.   ]
 [ 0.     0.     7.314  3.167]
 [ 0.     0.     0.    11.169]]


u is 1.1455 and d is 0.8966


##### b. Compute the premia of American and European calls and puts.

In [149]:
putPrcA = americanBinomPricerRecursive(S, K, r, v, q, T, n, putPayoff, verbose = True)
print(f"The American Put Premium is: {putPrcA : 0.4f}")
print(f"The American Call Premium is: {callPrc : 0.4f}")
print(f"The European Put Premium is: {putPrc : 0.4f}")
print(f"The European Call Premium is: {callPrc : 0.4f}")

[ 0.          0.          3.16660277 11.16886174]
[ 0.          0.          3.16660277 11.16886174]
[ 0.          1.65788424  3.16660277 11.16886174]
[ 0.          1.65788424  7.84406148 11.16886174]
[ 0.86799019  1.65788424  7.84406148 11.16886174]
[ 0.86799019  4.8747172   7.84406148 11.16886174]
[ 2.95422756  4.8747172   7.84406148 11.16886174]
The American Put Premium is:  2.9542
The American Call Premium is:  4.3774
The European Put Premium is:  2.8090
The European Call Premium is:  4.3774
